In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mat

#import psycopg2

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import datetime
import scipy.stats

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
''''df2 = pd.read_csv("/Users/GuntherUlvanget/country_life/country_life_spins/Sprouts_INFRA_24 weeks_ending_7_12_20.csv", 
                     sep = ",", na_values = "-",
                 usecols= [  'Geography', 'Item_UPC', 'Brand', 'Subcategory',
       'AGE', 'FORM', 'FUNCTIONAL_INGREDIENT', 'FLAVOR', 'HEALTH_FOCUS',
       'Group', 'SIZE', 'Dollars', 'Dollars_Yago', 'Units', 'Units_Yago', 'Dollars_SPP', 'ARP',
       'ARP_Yago',  'Base_ARP',
       'Base_ARP_Yago'])
       
       #df3 = process_data("/Users/GuntherUlvanget/country_life/country_life_spins/Sprouts_INFRA_24 weeks_ending_7_12_20.csv")'''

'\'df2 = pd.read_csv("/Users/GuntherUlvanget/country_life/country_life_spins/Sprouts_INFRA_24 weeks_ending_7_12_20.csv", \n                     sep = ",", na_values = "-",\n                 usecols= [  \'Geography\', \'Item_UPC\', \'Brand\', \'Subcategory\',\n       \'AGE\', \'FORM\', \'FUNCTIONAL_INGREDIENT\', \'FLAVOR\', \'HEALTH_FOCUS\',\n       \'Group\', \'SIZE\', \'Dollars\', \'Dollars_Yago\', \'Units\', \'Units_Yago\', \'Dollars_SPP\', \'ARP\',\n       \'ARP_Yago\',  \'Base_ARP\',\n       \'Base_ARP_Yago\'])\n       \n       #df3 = process_data("/Users/GuntherUlvanget/country_life/country_life_spins/Sprouts_INFRA_24 weeks_ending_7_12_20.csv")'

In [56]:
def clean_CL_UPC(df):
	#to match items in SPINS data, CL starts with Leading zero

	df["UPC"] = df["UPC"].apply(str)
	df["UPC"] = df["UPC"].apply(lambda x : x[:-1])
	df["UPC"] = df["UPC"].apply(lambda x : "0" + x)
	column_name = "UPC"
	first = df.pop(column_name)
	df.insert(0, column_name, first)
	df["MSRP$"] = df["MSRP$"].str.lstrip('$')
	df["WSP$"] = df["WSP$"].str.lstrip('$')
	df["MSRP$"] = df["MSRP$"].apply(float)
	df["WSP$"] = df["WSP$"].apply(float)
	df["Unit Size"] = df["Unit Size"].apply(int)
    
    
def process_data(file_path_todata):
    df = pd.read_csv(file_path_todata, 
        sep = ",", 
        na_values = "-",
       usecols= [  'Geography', 'Item_UPC', 'Brand', 'Subcategory',
           'AGE', 'FORM', 'FUNCTIONAL_INGREDIENT', 'FLAVOR', 'HEALTH_FOCUS',
        'Group', 'SIZE', 'Dollars', 'Dollars_Yago', 'Units', 'Units_Yago', 'Dollars_SPP', 'ARP',
        'ARP_Yago',  'Base_ARP',
        'Base_ARP_Yago']
         )

    df.dropna(axis = 0, how = "all", inplace=True)

    


    return df

def clean_spins_upc(df):
	#take in a dataframe, expand the number of columns into a temp
	#datafram
	temp = df["Item_UPC"].str.rsplit(" ", n=1, expand = True)
	#add columns  to main dataframe from Temp
	df.insert(0, "UPC", temp[1])
	df.insert(1, "Item", temp[0])

	#drop original combined column
	df.drop(columns= ["Item_UPC"], inplace=True)
	#SPINS data adds a leading zero to UP and includes hypen. 
	#neither are needed for analysis.
	df["UPC"]= df["UPC"].apply(lambda x : x[1:] if x.startswith("0") else x)
	df["UPC"]=df["UPC"].apply(lambda x :x.replace('-', ''))
	col_names = df.columns
	for cols in col_names:
	    df[cols] = df[cols].apply(lambda x: x.lower() if type(x) == str else x)
        
def sku_selcetor_ingredient(FUNCTIONAL_INGREDIENT, df):
	'''Functional ingredient can be a shortened description like cal for calcium
	or mag for magnesium. this will select all Items that contain the string entered'''
	cl_df = df[df["Brand"].str.contains("country life")]
	temp_df = cl_df[cl_df["Item"].str.contains(FUNCTIONAL_INGREDIENT)]
	
	return temp_df


def sku_singleton(item_id,df, brand = "country life"):
    '''Select a single SKU to examine, item ID can be either full UPC or catalog ID
    df = the dataframe you are subsetting'''
    df = df[df["Brand"].str.contains(brand)]
    df = df[df["UPC"].str.contains(item_id)].reset_index()
    #df["ARP_percent_off_MSRP"] =  df["ARP"].apply(lambda x: (1-(x/get_MSRP("item_id")))*100)
    df.loc[:, "ARP_percent_off_MSRP"] =  df.loc[:,"ARP"].apply(lambda x: (1-(x/get_MSRP(item_id)))*100)
    #df['bins'] = pd.cut(df['ARP_percent_off_MSRP'],bins=[0,5,10,15,20,25,30,35,40], labels=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%", "30-35%","35-40"])
    df.loc[:,'bins'] = pd.cut(df['ARP_percent_off_MSRP'],bins=[0,5,10,15,20,25,30,35,40], labels=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%", "30-35%","35-40"])
    
    return df


def get_MSRP(item_id ):
	temp = cl_top_seller[cl_top_seller.loc[:,"UPC"].str.contains(item_id)]
	return temp["MSRP$"][0]


def sub_set_metrics_of_interest(df, metric_to_use = ['ARP',"Base_ARP"]):
	return df.loc[df["Metric"].isin(metric_to_use)].reset_index()

def easy_melt(df):
	return df.melt(id_vars=["UPC", "Item","Geography",'Brand', 'Subcategory'], var_name = "Metric" , value_name='amount')

def get_discount_proportions(df):
    ''' Takes in a dataframe, returns a smaller dataframe'''
    group_by_df =df.groupby("bins").size()
    #groups are stored in the index, values in series
    #extract groups
    groups = pd.DataFrame(group_by_df).index
    final = group_by_df.to_frame().join(groups.to_frame())
    #rename column 0 for extracting data in pie chart
    final.rename(columns = {0:"count"}, inplace = True )
    return final

In [57]:
cl_top_seller = pd.read_csv("/Users/GuntherUlvanget/country_life/country_life_spins/Country Life Top Seller's Price File.csv", sep = ",")
clean_CL_UPC(cl_top_seller)


In [58]:
df = pd.read_csv("/Users/GuntherUlvanget/country_life/country_life_spins/Retailer_Report_24 Weeks_7_12_2020.csv", 
                 sep = ",", na_values="-", usecols= [ 'Geography', 'Item UPC', 'Brand', 'Subcategory',
       'Dollars', 'Dollars, Yago', 'Units', 'Units, Yago', 'Dollars SPP', 'Dollars SPP, Yago',  'ARP',
       'ARP, Yago',   'Base ARP','Base ARP, Yago'])
df.rename(columns =  {'Item UPC': 'Item_UPC', 
                      'Dollars, Yago':'Dollars_Yago', 
                      'Units, Yago':'Units_Yago', 
                      'Dollars SPP':'Dollars_SPP', 
                      'Dollars SPP, Yago': 'Dollars_SPP_Yago', 
                      'ARP, Yago': 'ARP_Yago',  
                      'Base ARP': 'Base_ARP',
                      'Base ARP, Yago':'BaseARP_Yago'},
         inplace= True)

clean_spins_upc(df)


In [ ]:
mag_df3 = sku_singleton('2474',df)
pie_frame = get_discount_proportions(mag_df3)

In [90]:
type(pie_frame["count"])

pandas.core.series.Series

In [60]:
cl_top_seller.loc[0,]

UPC                        01579402474
Item#                             2474
Retail Description    MAGNESIUM 300 MG
Unit Size                           60
Form                          Vegicaps
MSRP$                            10.79
WSP$                               5.4
Category                Other Minerals
Name: 0, dtype: object

In [44]:
mag_df4 = easy_melt(mag_df3)

In [39]:
mag_df4

(20, 18)

In [ ]:


#df["ARP_percent_off_MSRP"] = df["ARP"].apply(lambda x: (1-(x/get_MSRP("2474"))))
#mag_df.head()
                                                      

#mag_df2 = sku_selcetor_ingredient("mag",df)
#mag_df3 = sku_singleton('2474', df)
#mag_df3.head()

#df["ARP_percent_off_MSRP"] =  df["ARP"].apply(lambda x: (1-(x/get_MSRP("2474")))*100)
#df['bins'] = pd.cut(df['ARP_percent_off_MSRP'],bins=[0,5,10,15,20,25,30,35,40], labels=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%", "30-35%","35-40"])


#mag_df3.reset_index(inplace = True)

#mag_df3.loc[:,"ARP_percent_off_MSRP"] = (1-(mag_df3["ARP"]/get_MSRP("2474"))) *100

#mag_df3['bins'] = pd.cut(mag_df3['ARP_percent_off_MSRP'],bins=[0,5,10,15,20,25,30,35,40], labels=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%", "30-35%","35-40"])





In [61]:
pie_frame = get_discount_proportions(mag_df3)
pie_frame

,count,bins
bins,,
0-5%,1,0-5%
5-10%,2,5-10%
10-15%,0,10-15%
15-20%,2,15-20%
20-25%,12,20-25%
25-30%,2,25-30%
30-35%,1,30-35%
35-40,0,35-40


In [62]:
mag_df3 = easy_melt(mag_df3)

In [63]:
mag_df3.head(10)

,UPC,Item,Geography,Brand,Subcategory,Metric,amount
0,01579402474,country life magnesium 300 mg 60 vcp,greenacres - total us (corp),country life,minerals other,index,23451
1,01579402474,country life magnesium 300 mg 60 vcp,greens natural market - total us (corp),country life,minerals other,index,38117
2,01579402474,country life magnesium 300 mg 60 vcp,infra - total us (corp),country life,minerals other,index,46293
3,01579402474,country life magnesium 300 mg 60 vcp,natural grocers by vc - total us (corp),country life,minerals other,index,88879
4,01579402474,country life magnesium 300 mg 60 vcp,ncg - total us (corp),country life,minerals other,index,104553
5,01579402474,country life magnesium 300 mg 60 vcp,ncg - total us x-pcc (ban),country life,minerals other,index,130211
6,01579402474,country life magnesium 300 mg 60 vcp,ncg - pcc (ban),country life,minerals other,index,155513
7,01579402474,country life magnesium 300 mg 60 vcp,ncg - mid atlantic (rgn/mkt),country life,minerals other,index,158991
8,01579402474,country life magnesium 300 mg 60 vcp,ncg - north central (rgn/mkt),country life,minerals other,index,172172
9,01579402474,country life magnesium 300 mg 60 vcp,ncg - north east (rgn/mkt),country life,minerals other,index,188778


In [64]:
#mag_df3 = mag_df3.melt(id_vars=["UPC", "Item","Geography",'Brand', 'Subcategory'], var_name = "Metric" , value_name='amount')

In [65]:
mag_df3["Metric"].unique()

array(['index', 'Dollars', 'Dollars_Yago', 'Units', 'Units_Yago',
       'Dollars_SPP', 'Dollars_SPP_Yago', 'ARP', 'ARP_Yago', 'Base_ARP',
       'BaseARP_Yago', 'ARP_percent_off_MSRP', 'bins'], dtype=object)

In [66]:
def subset_for_boxplots(df):
    metric_to_use = ['ARP',"Base_ARP"]
    df = df[df["Metric"].isin(metric_to_use)].reset_index()

    return df

In [67]:
mag_df3 = subset_for_boxplots(mag_df3)
mag_df3

,index,UPC,Item,Geography,Brand,Subcategory,Metric,amount
0,140,01579402474,country life magnesium 300 mg 60 vcp,greenacres - total us (corp),country life,minerals other,ARP,8.15
1,141,01579402474,country life magnesium 300 mg 60 vcp,greens natural market - total us (corp),country life,minerals other,ARP,10.21
2,142,01579402474,country life magnesium 300 mg 60 vcp,infra - total us (corp),country life,minerals other,ARP,8.46
3,143,01579402474,country life magnesium 300 mg 60 vcp,natural grocers by vc - total us (corp),country life,minerals other,ARP,7.45
4,144,01579402474,country life magnesium 300 mg 60 vcp,ncg - total us (corp),country life,minerals other,ARP,8.56
5,145,01579402474,country life magnesium 300 mg 60 vcp,ncg - total us x-pcc (ban),country life,minerals other,ARP,8.35
6,146,01579402474,country life magnesium 300 mg 60 vcp,ncg - pcc (ban),country life,minerals other,ARP,10.39
7,147,01579402474,country life magnesium 300 mg 60 vcp,ncg - mid atlantic (rgn/mkt),country life,minerals other,ARP,8.7
8,148,01579402474,country life magnesium 300 mg 60 vcp,ncg - north central (rgn/mkt),country life,minerals other,ARP,8.54
9,149,01579402474,country life magnesium 300 mg 60 vcp,ncg - north east (rgn/mkt),country life,minerals other,ARP,8.32


In [69]:
def cl_box_plot(df):
    fig = px.box(df, x="Metric", y="amount", points="all", title = df["Item"][0])
    fig.add_trace(go.Scatter(x=['ARP', 'Base_ARP'], y=[get_MSRP(df.loc[0,"UPC"]),get_MSRP(df.loc[0,"UPC"])], mode="lines", name="MSRP"))
    fig.show()
    

In [ ]:
mag_df3 = mag_df3.loc[mag_df3["Metric"].isin(metric_to_use)].reset_index()

In [ ]:
ax = sns.boxplot(x='Metric', y='amount', data=mag_df3, showfliers = False)
ax = sns.stripplot(x='Metric', y='amount', data=mag_df3, color = "black", jitter = .1 )
ax.axhline(cl_top_seller["MSRP$"][0])
#plt.title(mag_df3["Item"][0])

In [71]:
mag_df3.loc[0,"UPC"]

'01579402474'

In [ ]:
fig = px.box(mag_df3, x="Metric", y="amount", points="all", title = mag_df3["Item"][0])
fig.add_trace(go.Scatter(x=['ARP', 'Base_ARP'], y=[get_MSRP("2474"),get_MSRP("2474")], mode="lines", name="MSRP"))
fig.show()

In [68]:
get_MSRP("01579402474")

10.79

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "polar"}],
           [{"type": "domain"}, {"type": "scene"}]],
)

fig.add_trace(go.Bar(y=[2, 3, 1]),
              row=1, col=1)

fig.add_trace(go.Barpolar(theta=[0, 45, 90], r=[2, 3, 1]),
              row=1, col=2)

fig.add_trace(go.Pie(values=[2, 3, 1]),
              row=2, col=1)

fig.add_trace(go.Scatter3d(x=[2, 3, 1], y=[0, 0, 0], 
                           z=[0.5, 1, 2], mode="lines"),
              row=2, col=2)

fig.update_layout(height=700, showlegend=False)

fig.show()

In [121]:
def make_plots(df,pie_frame):
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "box"}, {"type": "pie"}]],
    )

    fig.add_trace(go.Box( x=df.Metric, y=df.amount, boxpoints ="all", jitter =0.5,
                         boxmean = True, showlegend=False), row=1, col=1)
    #adds a line to reprsent 
    fig.add_trace(go.Scatter(x=['ARP', 'Base_ARP'], y=[get_MSRP(df.loc[0,"UPC"]),get_MSRP(df.loc[0,"UPC"])],
                             mode="lines", name="MSRP"))
    
    #generates pie chart from calculated proportion data
    fig.add_trace(go.Pie(values=pie_frame["count"], labels =pie_frame["bins"], showlegend=True,
                         title_text ="Proportion of ARP as a percentage discount off MSRP"),
                  row=1, col=2)
    # Update xaxis properties
    fig.update_xaxes(title_text="Distribution of ARP vs Base ARP", row=1, col=1)
    #fig.update(title_text="Proportion of ARP as percentage discount",  row=1, col=2)

    fig.update_yaxes(title_text="Dollars", row=1, col=1)
    fig.update_yaxes(title_text="Proportion of ARP as percentage discount",  row=1, col=2)

    #label the plot
    fig.update_layout(title_text=mag_df3["Item"][0], height=700)
    fig.update_layout(height=700, showlegend=True)

    fig.show()

In [122]:
make_plots(mag_df3,pie_frame)

In [123]:
df.head()

,UPC,Item,Geography,Brand,Subcategory,Dollars,Dollars_Yago,Units,Units_Yago,Dollars_SPP,Dollars_SPP_Yago,ARP,ARP_Yago,Base_ARP,BaseARP_Yago
0,65801011832,garden life dr frm prob onc dly wmn ogc 30vcp,earth fare - total us (corp),garden of life,prebiotics & probiotics,10579.14,47225.35,286.0,1442.0,182.87,486.06,36.99,32.75,36.99,36.06
1,85023200508,vital proteins collagen peptides 20 oz,earth fare - total us (corp),vital proteins,protein & meal replacement powder,10085.45,77767.14,205.0,1854.0,123.64,820.67,49.20,41.95,49.53,42.62
2,03367410486,nat way sambucus elderbrry gummies ogc 60 chw,earth fare - total us (corp),natures way,herbal formulas cold & flu & immune,7220.56,24308.32,344.0,1168.0,73.22,244.99,20.99,20.81,20.99,20.92
3,76899003790,nordic nat ultimate omega 180 sg,earth fare - total us (corp),nordic naturals,supplements oils,32276.56,43280.34,544.0,814.0,328.68,455.39,59.33,53.17,59.51,56.70
4,85810900611,ancient nutrition multi collagen complex 16oz,earth fare - total us (corp),ancient nutrition,protein & meal replacement powder,9986.64,51194.58,206.0,1050.0,106.24,533.05,48.48,48.76,48.81,49.00


In [ ]:
pie = px.pie(data_frame= c,
            names = "bins"
            , values = "count",
            title = "Breakdown of ARP as a percentage off MSRP")

pie.show()

pie.title.position 

In [ ]:
conn = psycopg2.connect("user='postgres' host='localhost'")

cur = conn.cursor()

conn.autocommit = True
cur.execute('DROP DATABASE IF EXISTS spins;')
cur.execute('CREATE DATABASE spins;')

cur.close() # This is optional
conn.close() # Closing the connection also closes all cursors

conn = psycopg2.connect("dbname='spins' user='postgres' host='localhost'")

cur = conn.cursor()

 # Inside insulinpricing database create a table with all of the 
    #appropriate column names to match with the CSV file header. '
query = '''
       
CREATE TABLE spins_data(Geography VARCHAR, 
Item_UPC, 
Brand, 
Subcategory,
AGE, 
FORM, 
FUNCTIONAL_INGREDIENT, 
FLAVOR, 
HEALTH_FOCUS,
Group, 
SIZE, 
Dollars, 
Dollars_Yago, 
Units, 
Units_Yago, 
Dollars_SPP, 
ARP,
ARP_Yago,  
Base_ARP,
Base_ARP_Yago)



(NDC_Description VARCHAR ,NDC VARCHAR, 
    NADAC_Per_Unit NUMERIC, Effective_Date DATE,
    Pricing_Unit VARCHAR,Pharmacy_Type_Indicator VARCHAR,
    OTC VARCHAR,Explanation_Code VARCHAR,
    Classification_for_Rate_Setting VARCHAR,
    Corresponding_Generic_Drug_NADAC_Per_Unit NUMERIC,
    Corresponding_Generic_Drug_Effective_Date DATE,
    As_of_Date DATE);
        '''
cur.execute(query)